In [2]:
! pip install pygame
import pygame
import librosa
import numpy as np
import time


def clamp(min_value, max_value, value):
    if value < min_value:
        return min_value
    if value > max_value:
        return max_value
    return value


class AudioBar:
    def __init__(self, x, y, freq, color, width=50, min_height=10, max_height=100, min_decibel=-80, max_decibel=0):
        self.x, self.y, self.freq = x, y, freq
        self.color = color
        self.width, self.min_height, self.max_height = width, min_height, max_height
        self.height = min_height
        self.min_decibel, self.max_decibel = min_decibel, max_decibel
        self.__decibel_height_ratio = (self.max_height - self.min_height) / (self.max_decibel - self.min_decibel)

    def update(self, dt, decibel):
        desired_height = decibel * self.__decibel_height_ratio + self.max_height
        speed = (desired_height - self.height) / 0.1
        self.height += speed * dt
        self.height = clamp(self.min_height, self.max_height, self.height)

    def render(self, screen):
        pygame.draw.rect(screen, self.color, (self.x, self.y + self.max_height - self.height, self.width, self.height))

# 定義兩個音訊文件的路徑
filename1 = r"C:\Users\garyhu\Desktop\鋼琴.mp3"
filename2 = r"C:\Users\garyhu\Desktop\提琴.mp3"

# 從第一個文件中獲取音訊信息
time_series1, sample_rate1 = librosa.load(filename1)
time_series2, sample_rate2 = librosa.load(filename2)

# 將音訊轉換為短時傅立葉變換（STFT）表示
stft1 = np.abs(librosa.stft(time_series1, hop_length=512, n_fft=2048 * 4))
stft2 = np.abs(librosa.stft(time_series2, hop_length=512, n_fft=2048 * 4))

# 將STFT矩陣轉換為分貝矩陣
spectrogram1 = librosa.amplitude_to_db(stft1, ref=np.max)
spectrogram2 = librosa.amplitude_to_db(stft2, ref=np.max)

# 獲取兩個文件的FFT頻率
frequencies = librosa.core.fft_frequencies(n_fft=2048 * 4)

# 獲取時間軸
times1 = librosa.core.frames_to_time(np.arange(spectrogram1.shape[1]), sr=sample_rate1, hop_length=512, n_fft=2048 * 4)
times2 = librosa.core.frames_to_time(np.arange(spectrogram2.shape[1]), sr=sample_rate2, hop_length=512, n_fft=2048 * 4)

# 計算時間索引比率
time_index_ratio1 = len(times1) / times1[len(times1) - 1]
time_index_ratio2 = len(times2) / times2[len(times2) - 1]


def get_decibel1(target_time, freq, spectrogram1, time_index_ratio1):
    return spectrogram1[int(freq * len(spectrogram1) / len(frequencies))][int(target_time * time_index_ratio1)]
def get_decibel2(target_time, freq, spectrogram2, time_index_ratio2):
    return spectrogram2[int(freq * len(spectrogram2) / len(frequencies))][int(target_time * time_index_ratio2)]
#def get_decibel(target_time, freq, spectrogram, time_index_ratio):
#    return spectrogram[int(freq * len(spectrogram) / frequencies[-1])][int(target_time * time_index_ratio)]
#def get_decibel(target_time, freq, spectrogram, time_index_ratio):
#    return spectrogram[int(freq * len(spectrogram) / len(frequencies))][int(target_time * time_index_ratio)]
#def get_decibel(target_time, freq, spectrogram, time_index_ratio):
#    return spectrogram[int(freq * len(spectrogram) / frequencies[-1])][int(target_time * time_index_ratio)]


print("spectrogram1 shape:", spectrogram1.shape)
print("spectrogram2 shape:", spectrogram2.shape)
print("spectrogram2:", spectrogram2)
print("frequencies shape:", frequencies.shape)
pygame.init()

infoObject = pygame.display.Info()

screen_w = int(infoObject.current_w / 2.5) * 2
screen_h = int(infoObject.current_w / 2.5)

# Set up the drawing window
screen = pygame.display.set_mode([screen_w, screen_h])

left_bars = []
right_bars = []

frequencies = np.arange(100, 1200, 10)
num_bars = len(frequencies)
width = screen_w / (num_bars * 2)  # 將畫面寬度分成左右兩半

x = (screen_w - width * num_bars) / 4  # 調整 x 起始位置，使兩邊的 bars 居中

# 在左側和右側分別建立 AudioBar 物件
for freq in frequencies:
    left_bars.append(
        AudioBar(x, 300, freq, (255, 0, 0), max_height=400, width=width, min_decibel=-80, max_decibel=0))
    right_bars.append(
        AudioBar(screen_w / 2 + x, 300, freq, (255, 0, 0), max_height=400, width=width, min_decibel=-80,
                 max_decibel=0))
    x += width

t = pygame.time.get_ticks()
getTicksLastFrame = t

pygame.mixer.music.load(filename1)
pygame.mixer.music.play(0)

# Run until the user asks to quit
running = True
while running:
    t = pygame.time.get_ticks()
    deltaTime = (t - getTicksLastFrame) / 1000.0
    getTicksLastFrame = t

    # Did the user click the window close button?
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    # Fill the background with white for the first window
    screen.fill((255, 255, 255))
    pygame.draw.line(screen, (0, 0, 0), (screen_w // 2, 0), (screen_w // 2, screen_h))
    # 更新和渲染左側的 bars
    for b in left_bars+right_bars:
        b.update(deltaTime, get_decibel1(pygame.mixer.music.get_pos() / 1000.0, b.freq, spectrogram1, time_index_ratio1))
        b.render(screen)
    time.sleep(1)

    # 更新和渲染右側的 bars，使用 filename2 的音樂資料
    #for b in right_bars:
    #    b.update(deltaTime,
    #             get_decibel2(pygame.mixer.music.get_pos() / 1000.0, b.freq, spectrogram2, time_index_ratio2))
    #    b.render(screen)

    # Flip the display for both windows
    pygame.display.flip()

# Done! Time to quit.
pygame.quit()


Defaulting to user installation because normal site-packages is not writeable
spectrogram1 shape: (4097, 183)
spectrogram2 shape: (4097, 168)
spectrogram2: [[-80.       -80.       -80.       ... -79.51949  -78.69475  -78.47457 ]
 [-80.       -80.       -80.       ... -80.       -78.99545  -78.4246  ]
 [-80.       -80.       -80.       ... -78.657715 -78.09495  -78.11654 ]
 ...
 [-80.       -80.       -80.       ... -80.       -80.       -80.      ]
 [-80.       -80.       -80.       ... -80.       -80.       -80.      ]
 [-80.       -80.       -80.       ... -80.       -80.       -80.      ]]
frequencies shape: (4097,)


IndexError: index 4097 is out of bounds for axis 0 with size 4097

: 